[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/Mix-of-Show-colab/blob/main/Mix_of_Show_multi_anime_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/Mix-of-Show
%cd /content/Mix-of-Show

!pip install -q diffusers==0.20.2 transformers accelerate einops omegaconf

# !mkdir -p /content/Mix-of-Show/models/1001_1_EDLoRA_hina_Anyv4_B4_Repeat500
# !wget https://huggingface.co/camenduru/Mix-of-Show/resolve/main/single_concept_model/1001_1_EDLoRA_hina_Anyv4_B4_Repeat500/models/edlora_model-latest.pth?download=true -O /content/Mix-of-Show/models/1001_1_EDLoRA_hina_Anyv4_B4_Repeat500/edlora_model-latest.pth
# !mkdir -p /content/Mix-of-Show/models/1002_1_EDLoRA_kaori_Anyv4_B4_Repeat500
# !wget https://huggingface.co/camenduru/Mix-of-Show/resolve/main/single_concept_model/1002_1_EDLoRA_kaori_Anyv4_B4_Repeat500/models/edlora_model-latest.pth?download=true -O /content/Mix-of-Show/models/1002_1_EDLoRA_kaori_Anyv4_B4_Repeat500/edlora_model-latest.pth
# !mkdir -p /content/Mix-of-Show/models/1003_1_EDLoRA_tezuka_Anyv4_B4_Repeat500
# !wget https://huggingface.co/camenduru/Mix-of-Show/resolve/main/single_concept_model/1003_1_EDLoRA_tezuka_Anyv4_B4_Repeat500/models/edlora_model-latest.pth?download=true -O /content/Mix-of-Show/models/1003_1_EDLoRA_tezuka_Anyv4_B4_Repeat500/edlora_model-latest.pth

# !mkdir -p /content/Mix-of-Show/multi-concept

# !python gradient_fusion.py \
#     --concept_cfg="/content/Mix-of-Show/datasets/data_cfgs/MixofShow/multi-concept/anime/hina+kario+tezuka_anythingv4.json" \
#     --save_path="/content/Mix-of-Show/multi-concept" \
#     --pretrained_models="xyn-ai/anything-v4.0" \
#     --optimize_textenc_iters=500 \
#     --optimize_unet_iters=50

import gdown
gdown.download_folder('https://drive.google.com/drive/folders/1OEWohYb509FPMeJluxWjXnWNG_49MJ4d', quiet=True)

import json
import os
import torch
from diffusers import DPMSolverMultistepScheduler
from mixofshow.pipelines.pipeline_edlora import EDLoRAPipeline
pretrained_model_path = '/content/Mix-of-Show/hina+kario+tezuka_anythingv4/combined_model_base'
enable_edlora = True  # True for edlora, False for lora
pipe = EDLoRAPipeline.from_pretrained(pretrained_model_path, scheduler=DPMSolverMultistepScheduler.from_pretrained(pretrained_model_path, subfolder='scheduler'), torch_dtype=torch.float16).to('cuda')
with open(f'{pretrained_model_path}/new_concept_cfg.json', 'r') as fr:
    new_concept_cfg = json.load(fr)
pipe.set_new_concept_cfg(new_concept_cfg)

In [ ]:
TOK = '<hina1> <hina2>'  # the TOK is the concept name when training lora/edlora
prompt = f'a {TOK} in front of mount fuji'
negative_prompt = 'longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'
image = pipe(prompt, negative_prompt=negative_prompt, height=1024, width=512, num_inference_steps=50, generator=torch.Generator('cuda'), guidance_scale=7.5).images[0]
image.save(f'res.jpg')
image

In [ ]:
%cd /content/Mix-of-Show

fused_model="/content/Mix-of-Show/hina+kario+tezuka_anythingv4/combined_model_base"
expdir="/content/Mix-of-Show/hina+kario+tezuka_anythingv4"

keypose_condition='/content/Mix-of-Show/datasets/validation_spatial_condition/multi-characters/anime_pose_2x/hina_tezuka_kario_2x.png'
keypose_adaptor_weight=1.0
sketch_condition=''
sketch_adaptor_weight=1.0

context_prompt='two girls and a boy are standing near a forest'
context_neg_prompt='longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'

region1_prompt='[a <hina1> <hina2>, standing near a forest]'
region1_neg_prompt="[${context_neg_prompt}]"
region1='[12, 36, 1024, 600]'

region2_prompt='[a <tezuka1> <tezuka2>, standing near a forest]'
region2_neg_prompt="[${context_neg_prompt}]"
region2='[18, 696, 1024, 1180]'

region5_prompt='[a <kaori1> <kaori2>, standing near a forest]'
region5_neg_prompt="[${context_neg_prompt}]"
region5='[142, 1259, 1024, 1956]'

prompt_rewrite=f"{region1_prompt}-*-{region1_neg_prompt}-*-{region1}|{region2_prompt}-*-{region2_neg_prompt}-*-{region2}|{region5_prompt}-*-{region5_neg_prompt}-*-{region5}"

print(prompt_rewrite)

!python regionally_controlable_sampling.py \
  --pretrained_model="{fused_model}" \
  --sketch_adaptor_weight="{sketch_adaptor_weight}" \
  --sketch_condition="{sketch_condition}" \
  --keypose_adaptor_weight="{keypose_adaptor_weight}" \
  --keypose_condition="{keypose_condition}" \
  --save_dir="{expdir}" \
  --prompt="{context_prompt}" \
  --negative_prompt="{context_neg_prompt}" \
  --prompt_rewrite="{prompt_rewrite}" \
  --suffix="baseline" \
  --seed=19